In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from sklearn.metrics import mean_absolute_error
from solar_forecasting.data_sources.get_data import download_file

In [37]:
# Testing out python script
download_file(download_to_disk = True)

Downloaded storage object X_train_copernicus.npz from bucket train_data_solar_project to local file ../raw_data/data.npz.


In [38]:
# Testing out python script with different arguments
download_file(
    download_to_disk = True, 
    destination_file_name = '../raw_data/baseline.npz')

Downloaded storage object X_train_copernicus.npz from bucket train_data_solar_project to local file ../raw_data/baseline.npz.


In [40]:
data = np.load('../raw_data/baseline.npz')

In [41]:
data.files

['datetime', 'GHI', 'CLS', 'SZA', 'SAA']

In [42]:
G = data['GHI'][:300]
C = data['CLS'][:300]
Z = data['SZA'][:300]
A = data['SAA'][:300]

In [22]:
# Checking if values are np.float64 and stat is

total_ghi = 0
stat_ghi = []

for x in range(G.shape[0]):
    for y in range(G.shape[1]):
        for z in range(G.shape[2]):
            for t in range(G.shape[3]):
                stat_ghi.append(G[x,y,z,t])
                if type(G[x,y,z,t]) != np.float64:
                    total_ghi+=1
total_ghi

0

In [23]:
total_cls = 0
stat_cls = []

for x in range(C.shape[0]):
    for y in range(C.shape[1]):
        for z in range(C.shape[2]):
            for t in range (C.shape[3]):
                stat_cls.append(C[x,y,z,t])
                if type(C[x,y,z,t]) != np.float64:
                    total_cls+=1
total_cls

0

In [24]:
total_sza = 0
stat_sza = []

for x in range(Z.shape[0]):
    for y in range(Z.shape[1]):
        for z in range(Z.shape[2]):
            for t in range(Z.shape[3]):
                stat_sza.append(Z[x,y,z,t])
                if type(Z[x,y,z,t]) != np.float64:
                    total_sza+=1
total_sza

0

In [25]:
total_saa = 0
stat_saa = []

for x in range(A.shape[0]):
    for y in range(A.shape[1]):
        for z in range(A.shape[2]):
            for t in range(A.shape[3]):
                stat_saa.append(A[x,y,z,t])
                if type(A[x,y,z,t]) != np.float64:
                    total_saa+=1
total_saa

0

In [26]:
# Create feature dataframes

df_ghi = pd.DataFrame(stat_ghi, columns=['GHI'])
df_cls = pd.DataFrame(stat_cls, columns=['CLS'])
df_sza = pd.DataFrame(stat_sza, columns=['SZA'])
df_saa = pd.DataFrame(stat_saa, columns=['SAA'])

In [27]:
len(df_ghi)

7873200

In [28]:
len(df_cls)

15746400

In [29]:
df_ghi.head(2)

,GHI
0,88.29
1,88.66


In [30]:
df_cls.head(2)

,CLS
0,99.87
1,100.27


In [36]:
# Create lagged dataset

# values_ghi = pd.Series(df_ghi.values)
lagged_ghi = pd.concat([df_ghi.shift(1), df_ghi], axis=1)
lagged_ghi.columns = ['t-1', 't+1']
lagged_ghi.head(5)

,t-1,t+1
0,NaN,88.29
1,88.29,88.66
2,88.66,89.00
3,89.00,89.38
4,89.38,89.68


In [48]:
from sklearn.model_selection import train_test_split

# Split into train and test sets

X_train = np.load('../raw_data/X_train_copernicus.npz')
X_test = np.load('../raw_data/X_test_copernicus.npz')
y_test = np.genfromtxt('../raw_data/y_train_zRvpCeO_nQsYtKN.csv')

In [50]:
from sklearn.metrics import mean_squared_error

def model_persistence(X):
    return X

# Validation

predictions = list()

for x in X_test:
	y_hat = model_persistence(x)
	predictions.append(y_hat)
 
test_score = mean_squared_error(y_test, predictions)
print('Test MSE: %.3f' % test_score)

ValueError: Found input variables with inconsistent numbers of samples: [1846, 5]